# Computing FLOPS, latency and fps of a model

It is important to have an idea of how to measure a video model's speed, so that you can choose the model that suits best for your use case.
In this tutorial, we provide two simple scripts to help you compute (1) FLOPS, (2) number of parameters, (3) fps and (4) latency.
These four numbers will help you evaluate the speed of this model.
To be specific, FLOPS means floating point operations per second, and fps means frame per second.
In terms of comparison, (1) FLOPS, the lower the better,
(2) number of parameters, the lower the better,
(3) fps, the higher the better,
(4) latency, the lower the better.


In [1]:
import os, time, random
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import numpy as np
import cv2
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from keras_flops import get_flops
from glob import glob
from tqdm import tqdm
import sys; sys.path.insert(0, '..')
from lib.metrics import jaccard, dice_coef,bce_dice_loss_new

/home/mpanta1/.conda/envs/tf_gpu/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
def prediction(test_ids, image_path, mask_path, model):
    # empty list to store results
    time_taken = []
    
    image_path = os.path.abspath(image_path)
    mask_path = os.path.abspath(mask_path)

    # Iterating through each image in test data
    for i in tqdm(test_ids, total=len(test_ids)):
        filename = i
        
        i = os.path.join(image_path, i)
        
        # Load image
        img = cv2.imread(i, cv2.IMREAD_COLOR)
        H, W, _ = img.shape

        img = cv2.resize(img, (512,512))
        img = img / 255.0
        img = np.expand_dims(img, axis=0) ## (1, 256, 256, 3)
        img = img.astype(np.float32)
    
        # Make prediction of mask
        
        start_time = time.time()
        prediction = model.predict(img, verbose=0)[0]
        eval_time = time.time() - start_time
        time_taken.append(eval_time)
        
        # Save Mask
        #mask = cv2.resize(prediction, (512, 512)) # (W, H)
        #mask = mask > 0.5
        #mask = mask * 255
        #mask = mask.astype(np.float32)
        #save_path = f"{save_dir}/{filename}.png"
        #cv2.imwrite(save_path, mask)
    mean_time = np.mean(time_taken)
    mean_fps = 1/mean_time
    print(f"Mean Time: {mean_time:1.7f} -Mean FPS: {mean_fps:1.7f}")
    return mean_time, mean_fps


In [3]:
def load_model(model_path, model_name, pca_layer=False):
    
    path = os.path.join(model_path, str(model_name), "best_model.h5")
    if pca_layer:
        tf.keras.backend.clear_session()
        with tf.keras.utils.CustomObjectScope({'jaccard':jaccard}, {'dice_coef':dice_coef}, {'tversky':'tversky'},
                                               {'bce_dice_loss_new': bce_dice_loss_new}, {'PCALayer': PCALayer}):
            model = tf.keras.models.load_model(path)  
        print(f'{model_name} is loaded')
        return model
        
    else:

        tf.keras.backend.clear_session()
        with tf.keras.utils.CustomObjectScope({'jaccard':jaccard}, {'dice_coef':dice_coef}, 
                                               {'bce_dice_loss_new': bce_dice_loss_new}):
            model = tf.keras.models.load_model(path)
        print(f'{model_name} is loaded')
        return model


In [4]:
img_height = 512
img_width = 512
root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
print(root_dir)
path_test_levee = os.path.join(root_dir,  "datasets", "test_images", "test") #exp2_test, pothole_test, with_grassland, #no_grassland inside "test_images"
test_images = sorted(next(os.walk(path_test_levee + "/images"))[2])

random.Random(6464).shuffle(test_images)

image_path = path_test_levee + "/images"
mask_path = path_test_levee + "/masks"
model_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
model_path = os.path.join(model_dir, "models", "IEEE_Access_Models_trained_on_6853images", "sandboilnet_variations_4e_4") #baseline_comparision_models_4e_4
models = os.listdir(model_path)

/home/mpanta1/usace_workspace/sandboil_research


In [5]:
save_dir = "test_results"
if os.path.exists(save_dir):
    pass
else:
    os.makedirs(save_dir)


In [6]:
class PCALayer(tf.keras.layers.Layer):
    def __init__(self, n_components, **kwargs):
        super(PCALayer, self).__init__(**kwargs)
        self.n_components = n_components

    def build(self, input_shape):
        self.shape = input_shape
        self.input_dim = int(input_shape[-1])
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.input_dim, self.n_components), dtype="float32",
                                      initializer='glorot_uniform',
                                      trainable=False)

    def call(self, x):
        # Flatten the input tensor
        #x = tf.linalg.normalize(x,axis=-1)
        #print(x.shape)
        # assumption is that the feature vector is normalized
        #x = tf.math.l2_normalize(x, axis=-1)
        batch_size = tf.shape(x)[0]
        flattened = tf.reshape(x, [batch_size, -1, self.input_dim])
        
        # Compute the mean and subtract it from the input tensor
        mean = tf.reduce_mean(flattened, axis=1, keepdims=True)
        centered = flattened - mean
        

        # Compute the covariance matrix
        cov = tf.matmul(centered, centered, transpose_a=True) / tf.cast(tf.shape(flattened)[1] - 1, tf.float32)

        # Compute the eigenvalues and eigenvectors of the covariance matrix
        eigenvalues, eigenvectors = tf.linalg.eigh(cov)

        # Sort the eigenvectors based on the eigenvalues
        idx = tf.argsort(eigenvalues, axis=-1, direction='DESCENDING')
        top_eigenvectors = tf.gather(eigenvectors, idx, batch_dims=1, axis=-1)
        top_eigenvectors = top_eigenvectors[:, :, :self.n_components]

        # Transpose the eigenvectors to match the input shape
        top_eigenvectors = tf.transpose(top_eigenvectors, perm=[0, 1, 2])
        
        # Project centered data onto top principal components
        projected = tf.matmul(centered, top_eigenvectors)

        # Reshape projected data and return as output
        output_shape = tf.concat([tf.shape(x)[:-1], [self.n_components]], axis=0)
        #output = tf.reshape(projected, shape=(-1, *self.output_shape))
        output = tf.reshape(projected, output_shape)
        return output



    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.n_components,)

    def get_config(self):
        config = super(PCALayer, self).get_config()
        config.update({'n_components': self.n_components})
        return config

## 1. Calculate FLOPS (Floating-Points OPerations per Second)

In [7]:
def save_flops(model, model_name):
    flops = get_flops(model, batch_size=1)
    with open('flops.txt', 'a') as f:
        f.write(f"FLOPS for {model_name}: {flops / 10 ** 9:.03} G")
        f.write("\n")
        print(f"FLOPS for {model_name}: {flops / 10 ** 9:.03} G")

In [ ]:
model_names = [item for item in models if os.path.isdir(os.path.join(model_path, item))]
#model_names = ["SandBoilNet_Low_Dimension_PCA_bce_dice_loss_new"]
    
model_list = []
for model_name in model_names:
    model = load_model(model_path, model_name, pca_layer=True)
    save_flops(model, model_name)
    tf.keras.backend.clear_session()

In [10]:
model_names = [item for item in models if os.path.isdir(os.path.join(model_path, item))]
for model_name in model_names:
    model = load_model(model_path, model_name, pca_layer=True)
    mean_time, mean_fps = prediction(test_images, os.path.abspath(image_path), os.path.abspath(mask_path), model)
    with open('fps.csv', 'a') as f:
        f.write(f"{model_name},Mean Time: {mean_time:1.7f},Mean FPS: {mean_fps:1.7f}")
        f.write("\n")


SandBoilNet_4e_4_bce_dice_loss_new is loaded


100%|██████████| 51/51 [00:12<00:00,  4.15it/s]


Mean Time: 0.1571429 -Mean FPS: 6.3636346
SandBoilNet_SE_bce_dice_loss_new is loaded


100%|██████████| 51/51 [00:12<00:00,  4.12it/s]


Mean Time: 0.1577120 -Mean FPS: 6.3406697
baseline_normal_bce_dice_loss_new is loaded


...

Mean Time: 0.2261550 -Mean FPS: 4.4217460
SandBoilNet_Dropout_Without_PCA_bce_dice_loss_new is loaded


100%|██████████| 51/51 [00:11<00:00,  4.35it/s]


Mean Time: 0.1445217 -Mean FPS: 6.9193755
SandBoilNet_4e_4_noDropout_bce_dice_loss_new is loaded


100%|██████████| 51/51 [00:12<00:00,  4.02it/s]

Mean Time: 0.1627761 -Mean FPS: 6.1434096
